In [7]:
import torch
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import librosa
from tqdm import tqdm

In [3]:
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

In [4]:
high = 1
low = -1
sample_size = 10000
sequence_size = 1000
degree = 3
x = torch.linspace(-1, 1, sequence_size)
x = x.unsqueeze(1).expand(sequence_size, sample_size)
y = torch.zeros_like(x)
for n in range(degree):
    p = (high - low) * torch.rand(sample_size)
    y += p*(x**(n+1))

print(y.shape)

torch.Size([1000, 10000])


In [ ]:
'''y_np = y.numpy()
y_np = y_np/np.max(np.abs(y_np))
sample_y = y_np[:, 0]
D = librosa.stft(sample_y, n_fft=2032, hop_length=8)
S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
spectrogram = np.expand_dims(S_db, axis=0)
spectrogram = torch.tensor(spectrogram).to(device)
print(spectrogram.shape)'''

In [5]:
y_np = y.numpy().swapaxes(0,1)
y_np = y_np/np.max(np.abs(y_np))

def to_spectrogram(data, n_fft=2032, hop_length=8):
    D = librosa.stft(data, n_fft=n_fft, hop_length=hop_length)
    S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
    spectrogram = np.expand_dims(S_db, axis=0)
    return torch.tensor(spectrogram)

In [8]:
dataloader = DataLoader(y_np, batch_size=100)
all_spectrograms = []

for batch in tqdm(dataloader):
    batch = batch.numpy()
    batch_spectrograms = [to_spectrogram(s) for s in batch]
    batch_tensor = torch.vstack(batch_spectrograms)
    all_spectrograms.append(batch_tensor)    

  0%|          | 0/100 [00:00<?, ?it/s]/home/cymberly/miniconda3/envs/multifunc/lib/python3.11/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2032 is too large for input signal of length=1000
  warnings.warn(
100%|██████████| 100/100 [06:27<00:00,  3.87s/it]


In [ ]:
all_spectrograms = torch.vstack(all_spectrograms).to(device)
all_spectrograms.shape

In [ ]:
start = 0
end = int(all_spectrograms.size(0))

for p in range(10):
    torch.save(all_spectrograms[start:end], f"spectrograms{p}.pt")
    start = end
    end += int(all_spectrograms.size(0))

In [9]:
torch.save(x, "x_data.pt")

In [ ]:
y_arrays = []
for o in range(10):
    y_arrays.append(torch.load(f"spectrograms{p}.pt"))
y_data = torch.vstack(y_arrays)
y_data.shape

In [11]:
x_data = torch.load("x_data.pt")
x_data.shape

/tmp/ipykernel_3865369/1378707812.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x_data = torch.load("x_data.pt")


torch.Size([1000, 10000])